In [1]:
import os
import json
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

# ---------- OpenAI ----------
client = OpenAI()
MODEL = "qwen/qwen3-next-80b-a3b-instruct"
print("DEBUG: OpenAI client initialized")
print("DEBUG: API key loaded:", bool(os.getenv("OPENAI_API_KEY")))

# ---------- SQLite ----------
DB_PATH = "hospital.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cur = conn.cursor()
print("DEBUG: SQLite connected ->", DB_PATH)

# ---------- Excel ----------
EXCEL_PATH = "hospital_data.xlsx"
print("DEBUG: Loading Excel file ->", EXCEL_PATH)

inventory_df = pd.read_excel(EXCEL_PATH, sheet_name="inventory")
print("DEBUG: inventory rows:", len(inventory_df))

machines_df = pd.read_excel(EXCEL_PATH, sheet_name="machines")
print("DEBUG: machines rows:", len(machines_df))

vendor_detail_df = pd.read_excel(EXCEL_PATH, sheet_name="vendor_detail")
print("DEBUG: vendor_detail rows:", len(vendor_detail_df))

patient_df = pd.read_excel(EXCEL_PATH, sheet_name="patient_list")
print("DEBUG: patient_list rows:", len(patient_df))


DEBUG: OpenAI client initialized
DEBUG: API key loaded: True
DEBUG: SQLite connected -> hospital.db
DEBUG: Loading Excel file -> hospital_data.xlsx
DEBUG: inventory rows: 40
DEBUG: machines rows: 20
DEBUG: vendor_detail rows: 10
DEBUG: patient_list rows: 10


In [2]:
# Create a cursor object to execute SQL commands on the SQLite database
import sqlite3

DB_PATH = "hospital.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cur = conn.cursor()

# STEP 1: Drop existing tables
# Ensures old data/schema is removed before inserting fresh Excel data

print("DEBUG: Dropping existing tables")
cur.execute("DROP TABLE IF EXISTS inventory")
cur.execute("DROP TABLE IF EXISTS machines")
cur.execute("DROP TABLE IF EXISTS vendor_detail")
cur.execute("DROP TABLE IF EXISTS patient_list")
conn.commit()

# STEP 2: Write Excel sheets into SQLite tables
# Each pandas DataFrame is converted into a SQLite table

print("DEBUG: Writing Excel → SQLite (hospital mapping)")
inventory_df.to_sql("inventory", conn, if_exists="replace", index=False)
machines_df.to_sql("machines", conn, if_exists="replace", index=False)
vendor_detail_df.to_sql("vendor_detail", conn, if_exists="replace", index=False)
patient_df.to_sql("patient_list", conn, if_exists="replace", index=False)

print("DEBUG: Tables written successfully")

# STEP 3: Verify table schemas
# Prints column names and types for each created table

for t in ["inventory", "machines", "vendor_detail", "patient_list"]:
    cur.execute(f"PRAGMA table_info({t})")
    print(f"\nDEBUG {t} columns:")
    for row in cur.fetchall():
        print(row)


DEBUG: Dropping existing tables
DEBUG: Writing Excel → SQLite (hospital mapping)
DEBUG: Tables written successfully

DEBUG inventory columns:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'item_name', 'TEXT', 0, None, 0)
(2, 'category', 'TEXT', 0, None, 0)
(3, 'quantity', 'INTEGER', 0, None, 0)
(4, 'price', 'INTEGER', 0, None, 0)
(5, 'location', 'TEXT', 0, None, 0)

DEBUG machines columns:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'machine_name', 'TEXT', 0, None, 0)
(2, 'location', 'TEXT', 0, None, 0)
(3, 'status', 'TEXT', 0, None, 0)
(4, 'last_maintenance_date', 'TIMESTAMP', 0, None, 0)
(5, 'next_maintenance_date', 'TIMESTAMP', 0, None, 0)
(6, 'purchase_date', 'TIMESTAMP', 0, None, 0)
(7, 'warranty_until', 'TIMESTAMP', 0, None, 0)
(8, 'capacity', 'TEXT', 0, None, 0)
(9, 'energy_usage_kw', 'REAL', 0, None, 0)
(10, 'supplier_vendor_id', 'TEXT', 0, None, 0)
(11, 'department', 'TEXT', 0, None, 0)
(12, 'equipment_type', 'TEXT', 0, None, 0)
(13, 'priority', 'TEXT', 0, None, 0)
(14, 'notes', 'TEXT', 0, No

In [3]:
from rapidfuzz import process, fuzz

# SCHEMA DESCRIPTION
# This text is sent to the LLM so it understands:
# - What tables exist
# - What columns each table has
# - What rules to follow while generating JSON

SCHEMA = """
Tables:
inventory(item_name, category, quantity, price, location)
machines(id, machine_name, location, status, last_maintenance_date, next_maintenance_date, purchase_date, warranty_until, capacity, energy_usage_kw, supplier_vendor_id, department, equipment_type, priority, notes)
vendor_detail(vendor_id, vendor_name, contact_person, phone, email, primary_items_supplied)
patient_list(patient_id, name, age, gender, admission_date, discharge_date, department, room_bed, primary_diagnosis, secondary_diagnosis, attending_doctor, status, blood_type, allergies)

Rules:
- Output ONLY JSON
- Keys: action, table, columns, filters, group_by, limit
- Use ONLY listed tables & columns
- Do NOT generate SQL
- LIKE means "text contains value"
"""


# FEW-SHOT EXAMPLES
# These examples teach the model how to convert
# natural language questions into structured JSON

FEW_SHOTS = [
    {
        "user": "How many units of normal saline are available?",
        "json": {
            "action": "select",
            "table": "inventory",
            "columns": ["quantity"],
            "filters": [
                {"column": "item_name", "op": "LIKE", "value": "Normal Saline"}
            ],
            "group_by": [],
            "limit": None
        }
    },
    {
        "user": "List all critical machines in ICU",
        "json": {
            "action": "select",
            "table": "machines",
            "columns": ["machine_name", "status", "location"],
            "filters": [
                {"column": "priority", "op": "=", "value": "Critical"},
                {"column": "department", "op": "LIKE", "value": "ICU"}
            ],
            "group_by": [],
            "limit": None
        }
    },
    {
        "user": "Who supplies IV fluids?",
        "json": {
            "action": "select",
            "table": "vendor_detail",
            "columns": ["vendor_name", "contact_person", "phone"],
            "filters": [
                {"column": "primary_items_supplied", "op": "LIKE", "value": "IV"}
            ],
            "group_by": [],
            "limit": None
        }
    },
    {
        "user": "Show admitted patients in cardiology",
        "json": {
            "action": "select",
            "table": "patient_list",
            "columns": ["name", "room_bed", "primary_diagnosis"],
            "filters": [
                {"column": "department", "op": "=", "value": "Cardiology"},
                {"column": "status", "op": "=", "value": "Admitted"}
            ],
            "group_by": [],
            "limit": None
        }
    }
]


# QUERY CLASSIFIER
# Decides whether a user query is database-related
# This helps route queries to DB logic vs chat

def is_db_query(q):
    keywords = [
        # inventory / medical items
        "saline", "insulin", "antibiotic", "syringe", "gloves",
        "medicine", "drug", "inventory", "stock", "quantity",

        # machines / equipment
        "machine", "equipment", "ventilator", "mri", "ct",
        "maintenance", "warranty", "status",

        # patients
        "patient", "admitted", "discharged", "diagnosis",
        "ward", "icu", "department", "bed",

        # vendors
        "vendor", "supplier", "supplies", "contact", "phone", "email",

        # generic db intent
        "show", "list", "how many", "who", "available"
    ]
    return any(k in q.lower() for k in keywords)


In [4]:
DEPARTMENT_SYNONYMS = {
    "icu": "Critical Care",
    "intensive care": "Critical Care",
    "intensive care unit": "Critical Care",
    "critical care unit": "Critical Care"
}
STATUS_SYNONYMS = {
    "under maintenance": "Maintenance",
    "maintenance": "Maintenance",

    "under repair": "Under Repair",
    "repair": "Under Repair",
    "repairing": "Under Repair",

    "broken": "Out of Service",
    "not working": "Out of Service",
    "out of service": "Out of Service",

    "faulty": "Faulty",
    "old": "Outdated",
    "outdated": "Outdated",

    "active": "Active"
}


In [5]:
from rapidfuzz import process, fuzz

# Utility function to fetch distinct column values
def fetch_column_values(table, column):
    cur = conn.cursor()
    cur.execute(f"SELECT DISTINCT {column} FROM {table}")
    vals = [r[0] for r in cur.fetchall() if r[0] is not None]
    print(f"DEBUG Canonical {table}.{column}:", vals)
    return vals


# Canonical values from database
CANON_ITEMS = fetch_column_values("inventory", "item_name")
CANON_VENDORS = fetch_column_values("vendor_detail", "vendor_name")
CANON_VENDOR_CONTACTS = fetch_column_values("vendor_detail", "contact_person")

CANON_MACHINES = fetch_column_values("machines", "machine_name")
CANON_DEPARTMENTS = fetch_column_values("machines", "department")

CANON_PATIENTS = fetch_column_values("patient_list", "name")
CANON_DIAGNOSIS = fetch_column_values("patient_list", "primary_diagnosis")


# Fuzzy matching helper
def fuzzy_match(term, choices):
    if not term or not choices:
        return None

    match, score, _ = process.extractOne(
        str(term).lower(), choices, scorer=fuzz.WRatio
    )
    print(f"DEBUG fuzzy '{term}' → '{match}' ({score})")
    return match if score >= 70 else None


# Normalize intent filter values
def normalize_intent_values(intent):
    for f in intent.get("filters", []):
        m = None

        if f["column"] == "item_name":
            m = fuzzy_match(f["value"], CANON_ITEMS)

        elif f["column"] == "vendor_name":
            m = fuzzy_match(f["value"], CANON_VENDORS)

        elif f["column"] == "contact_person":
            m = fuzzy_match(f["value"], CANON_VENDOR_CONTACTS)

        elif f["column"] == "machine_name":
            m = fuzzy_match(f["value"], CANON_MACHINES)

        elif f["column"] == "status":
            v = str(f["value"]).lower().strip()

            # Status normalization (HARD MAP)
            if v in STATUS_SYNONYMS:
                m = STATUS_SYNONYMS[v]
                print("DEBUG status normalized:", f["value"], "→", m)

        elif f["column"] == "department":
            v = str(f["value"]).lower().strip()

            # ICU / Critical Care normalization (HARD MAP)
            if v in DEPARTMENT_SYNONYMS:
                m = DEPARTMENT_SYNONYMS[v]
                print(
                    "DEBUG department normalized (synonym):",
                    f["value"], "→", m
                )
            else:
                m = fuzzy_match(f["value"], CANON_DEPARTMENTS)

        elif f["column"] == "name":
            m = fuzzy_match(f["value"], CANON_PATIENTS)

        elif f["column"] == "primary_diagnosis":
            m = fuzzy_match(f["value"], CANON_DIAGNOSIS)

        if m:
            print("DEBUG normalized:", f["value"], "→", m)
            f["value"] = m

    return intent


DEBUG Canonical inventory.item_name: ['Normal Saline (0.9%)', 'Dextrose Saline (5%)', 'Paracetamol 500mg', 'Ibuprofen 400mg', 'Ringer Lactate', 'Normal Saline (0.45%)', 'Adrenaline 1mg', 'Insulin Regular', 'Insulin NPH', 'Atropine 0.6mg', 'Sterile Gauze Pads', 'Bandages (Various Sizes)', 'Metformin 500mg', 'Amoxicillin 250mg', 'Distilled Water 500ml', 'Ceftriaxone 1g', 'Morphine 10mg', 'Ciprofloxacin 500mg', 'Ampicillin 500mg', 'Disposable Syringe 5ml', 'IV Cannula 20G', 'Disposable Syringe 10ml', 'Surgical Gloves (Medium)', 'Surgical Gloves (Large)', 'IV Set (Standard)', 'Dopamine 200mg', 'Emergency Kit (Trauma)', 'First Aid Kit (Basic)', 'Antiseptic Solution 100ml', 'Betadine Solution 100ml', 'Plaster Cast Material', 'Oxygen Mask', 'IV Fluids Combo Pack', 'Nebulizer Solution', 'Normal Saline for Irrigation', 'Sterile Water for Injection', 'Cotton Balls', 'Alcohol Swabs', 'Surgical Masks', 'Surgical Caps']
DEBUG Canonical vendor_detail.vendor_name: ['BioMed Supplies', 'PharmaCare', 'M

In [6]:
# Required keys that MUST be present in the intent JSON
# This is used to validate LLM output structure

REQUIRED_KEYS = {"action", "table", "columns", "filters", "group_by", "limit"}


# Converts a natural language user query into
# a structured intent (JSON) using the LLM

def query_to_intent(user_query):

    # Start prompt with database schema description
    # This tells the LLM what hospital tables and columns exist
    prompt = SCHEMA + "\n\n"

    # Add few-shot examples to guide the LLM
    # These examples teach the expected input → output format
    for ex in FEW_SHOTS:
        prompt += (
            f"User: {ex['user']}\n"
            f"Output: {json.dumps(ex['json'])}\n\n"
        )

    # Append the actual user query
    # Explicitly instruct the model to output ONLY JSON
    prompt += f"User: {user_query}\nOutput ONLY JSON."

    print("DEBUG: Sending to LLM-A")

    # Send prompt to the LLM
    # Temperature = 0 ensures deterministic output
    res = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # Extract raw text output and clean code fences if present
    raw = res.choices[0].message.content.replace("```", "").strip()
    print("DEBUG RAW INTENT:", raw)

    try:
        # Parse LLM output as JSON
        intent = json.loads(raw)

        # Validate that all required keys are present
        if REQUIRED_KEYS - intent.keys():
            raise ValueError("Missing keys")

    except Exception as e:
        # If parsing or validation fails, return unknown intent
        print("ERROR intent parse:", e)
        return {"action": "unknown"}

    # Normalize intent values using fuzzy matching
    # (e.g., item names, machine names, vendor names, patient data)
    intent = normalize_intent_values(intent)

    print("DEBUG FINAL INTENT:", intent)
    return intent


In [7]:
# -------------------------------------------------
# Mapping of table names to their corresponding
# pandas DataFrames loaded from Excel.
# This allows query execution directly on DataFrames
# instead of SQLite for fast lookups.
# -------------------------------------------------
DATA_TABLES = {
    "inventory": inventory_df,
    "machines": machines_df,
    "vendor_detail": vendor_detail_df,
    "patient_list": patient_df
}


# -------------------------------------------------
# Executes a structured intent on the Excel data
# using pandas operations.
# Supports:
# - table selection
# - equality-based filters
# - LIKE (contains) filters
# - column projection
# -------------------------------------------------
def run_excel_query(intent):
    print("DEBUG running query:", intent)

    df = DATA_TABLES.get(intent["table"])
    if df is None:
        print("DEBUG unknown table:", intent["table"])
        return []

    result = df.copy()

    for f in intent["filters"]:
        col = f["column"]
        val = str(f["value"])

        if f["op"] == "=":
            result = result[result[col] == f["value"]]

        elif f["op"] == "LIKE":
            print(f"DEBUG applying LIKE filter: {col} contains '{val}'")
            result = result[result[col].astype(str).str.contains(val, case=False, na=False)]

    if intent["columns"] != ["*"]:
        result = result[intent["columns"]]

    print("DEBUG result rows:", len(result))
    return result.values.tolist()


In [8]:
# -------------------------------------------------
# Fallback actions for each table
# Used when a query returns no data
# -------------------------------------------------
fallback_actions = {
    "inventory": "Check storage room or food counter.",
    "staff_orders": "Check manual logbook or HR.",
    "vendor_detail": "Contact procurement team or vendor records.",
    "default": "Information not available."
}


# -------------------------------------------------
# Evaluates the query result
# If no data is returned, signals a fallback
# -------------------------------------------------
def result_evaluator(result):
    if not result:
        return {"status": "fallback", "reason": "no_data"}
    return {"status": "ok"}


# -------------------------------------------------
# Generates a human-readable fallback message
# based on the table involved in the intent
# -------------------------------------------------
def get_fallback_message(intent, reason):
    table = intent.get("table")
    filters = intent.get("filters", [])

    # Build human-readable filter explanation
    conditions = []
    for f in filters:
        conditions.append(f"{f['column']} = '{f['value']}'")

    condition_text = " and ".join(conditions) if conditions else "the given criteria"

    if table == "machines":
        return f"No machines match {condition_text} in the current data."

    if table == "inventory":
        return f"No inventory items match {condition_text} in the current data."

    if table == "vendor_detail":
        return f"No vendors match {condition_text} in the current data."

    if table == "patient_list":
        return f"No patients match {condition_text} in the current data."

    return "No matching data was found for your request."



In [9]:
# -------------------------------------------------
# System prompt for the response-generation LLM
# This LLM explains results in natural language
# and provides guidance when data is missing
# -------------------------------------------------
SYSTEM_PREFIX = """
You are a Hospital Operations Assistant.

You have TWO modes of behavior:
Rules:
- Explain ONLY using the Database Result provided
- Do NOT add assumptions, medical advice, treatment plans, or suggestions
- Do NOT invent vendor capabilities, machine functions, or patient details
- If data is missing, clearly say it is missing

For casual chat (jokes, greetings):
- Respond naturally

1. If the user is asking about hospital operations
   (inventory, machines, vendors, patients):
   - Explain results clearly
   - Use only provided data
   - Do NOT invent information
   - If an item or machine is unavailable, mention the status
     and share vendor contact details ONLY if available in data

2. If the user is asking a casual or general question
   (jokes, greetings, small talk):
   - Respond naturally and helpfully like a normal chatbot
   - You MAY tell jokes or have light conversation
   - Do NOT force hospital data into the response

Tone:
- Friendly
- Clear
- Helpful
"""


In [10]:
# -------------------------------------------------
# Sends a message to the explanation LLM (LLM-B)
# Uses conversation history as context
# -------------------------------------------------
def chat_llm(msg, history=None):
    print("DEBUG: Sending to LLM-B")

    messages = [
        {"role": "system", "content": SYSTEM_PREFIX}
    ]

    # Add previous conversation turns (context)
    if history:
        messages.extend(history)

    # Current user message
    messages.append({"role": "user", "content": msg})

    res = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.6
    )

    return res.choices[0].message.content


In [11]:
# -------------------------------------------------
# Formats database query results into a structured
# explanation request for the LLM
# Includes intent so the LLM understands context
# -------------------------------------------------
def format_db_results(user_query, result, intent):
    return f"""
User Query:
{user_query}

Query Intent:
{intent}

Database Result:
{result}

Instructions:
- Explain the result clearly using ONLY the database result
- If the result is empty or indicates unavailability,
  mention that the data is missing or unavailable
- Share vendor contact details ONLY if present in the data
- Do NOT invent missing data
"""


In [12]:
DEBUG_LOGS = []
DEBUG_ENABLED = True

def debug(msg):
    if not DEBUG_ENABLED:
        return

    log = f"[DEBUG] {msg}"
    print(log)
    DEBUG_LOGS.append(log)

    # Keep last 50 debug messages only
    if len(DEBUG_LOGS) > 50:
        DEBUG_LOGS.pop(0)


In [13]:
# NOTE:
# Identity and casual questions are handled BEFORE database routing
# to prevent accidental data access and hallucinated responses.
# This ensures non-operational queries never touch hospital data logic.

def ask(user_query, history=None):
    debug(f"User Query → {user_query}")

    q = user_query.lower().strip()

    # ---- FIX: identity / system questions ----
    if q in [
        "who are you",
        "what are you",
        "what do you do",
        "who is this",
        "what is this system"
    ]:
        debug("Identity query detected → direct response")
        return "I am the Hospital Operations Assistant for internal management queries."

    # Step 1: If query is NOT database-related,
    # respond directly using the chat LLM
    if not is_db_query(user_query):
        debug("Non-database query → routing to chat LLM")
        return chat_llm(user_query, history)

    # Step 2: Convert user query into structured intent (JSON)
    debug("Database query detected → generating intent")
    intent = query_to_intent(user_query)

    if intent.get("action") == "unknown":
        debug("Intent generation failed → unknown action")
        return "Sorry, I couldn't understand that."

    # Step 3: Execute the intent on Excel data
    debug(f"Executing query on table → {intent['table']}")
    result = run_excel_query(intent)

    # Step 4: Evaluate query result
    eval_out = result_evaluator(result)

    # Step 5: If data exists, ask LLM to explain results
    if eval_out["status"] == "ok":
        debug("Data found → sending result to explanation LLM")
        return chat_llm(format_db_results(user_query, result, intent), history)

    # Step 6: If no data is found, fallback
    debug("No data found → triggering fallback")
    return chat_llm(get_fallback_message(intent, eval_out["reason"]), history)


In [14]:
# -------------------------------------------------
# Extended test queries to validate:
# - Inventory queries
# - Machine & equipment queries
# - Vendor contact & supplier reasoning
# - Patient-related queries
# - Natural language (non-DB) handling
#
# While running, manually verify whether
# the answers match expected behavior.
# -------------------------------------------------
tests = [
    # -------- Inventory related --------
    "How many units of normal saline are available?",
    # Expected: Quantity from inventory OR unavailable message

    "Do we have insulin in stock?",
    # Expected: Inventory quantity for insulin types

    # -------- Machines / Equipment --------
    "List all critical machines in ICU",
    # Expected: Ventilator, oxygen-related equipment with status

    "Which machines are under maintenance?",
    # Expected: Autoclave or any machine with Maintenance status

    # -------- Vendor related (IMPORTANT) --------
    "Who supplies IV fluids?",
    # Expected: Vendor name + contact details from vendor_detail

    "How can I contact the emergency drug supplier?",
    # Expected: Vendor contact person + phone/email if available

    # -------- Patient related --------
    "Show admitted patients in cardiology",
    # Expected: Patient names, beds, diagnoses

    "Which patients are currently in ICU?",
    # Expected: ICU patient list OR empty result message

    # -------- Non-database query --------
    "Tell me a joke"
    # Expected: Normal chat response (not DB driven)
]

# -------------------------------------------------
# Execute each test query and print results
# Manually inspect answers for correctness
# -------------------------------------------------
for q in tests:
    print("\nQ:", q)
    print("A:", ask(q))



Q: How many units of normal saline are available?
[DEBUG] User Query → How many units of normal saline are available?
[DEBUG] Database query detected → generating intent
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["quantity"], "filters": [{"column": "item_name", "op": "LIKE", "value": "Normal Saline"}], "group_by": [], "limit": null}
DEBUG fuzzy 'Normal Saline' → 'Normal Saline (0.9%)' (76.15384615384616)
DEBUG normalized: Normal Saline → Normal Saline (0.9%)
DEBUG FINAL INTENT: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Normal Saline (0.9%)'}], 'group_by': [], 'limit': None}
[DEBUG] Executing query on table → inventory
DEBUG running query: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Normal Saline (0.9%)'}], 'group_by': [], 'limit': None}
DEBUG applying LIKE filter: 

C:\Users\ayush\AppData\Local\Temp\ipykernel_8524\3587826986.py:43: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result = result[result[col].astype(str).str.contains(val, case=False, na=False)]


A: No inventory items match item_name = 'Normal Saline (0.9%)' in the current data.

Q: Do we have insulin in stock?
[DEBUG] User Query → Do we have insulin in stock?
[DEBUG] Database query detected → generating intent
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["quantity"], "filters": [{"column": "item_name", "op": "LIKE", "value": "Insulin"}], "group_by": [], "limit": null}
DEBUG fuzzy 'Insulin' → 'Insulin Regular' (77.14285714285715)
DEBUG normalized: Insulin → Insulin Regular
DEBUG FINAL INTENT: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Insulin Regular'}], 'group_by': [], 'limit': None}
[DEBUG] Executing query on table → inventory
DEBUG running query: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Insulin Regular'}], 'group_by': [], 'limit': None}
DEBUG applying LIK

In [15]:
import gradio as gr

# -------------------------------------------------
# Wrapper function for Gradio chat UI
# -------------------------------------------------
def ask_chat(msg, history):
    reply = ask(msg, history)

    history.append({"role": "user", "content": msg})
    history.append({"role": "assistant", "content": reply})

    return history, ""


# -------------------------------------------------
# Helper to fetch debug logs
# -------------------------------------------------
def get_debug_logs():
    return "\n".join(DEBUG_LOGS)


# -------------------------------------------------
# Toggle debug mode
# -------------------------------------------------
def toggle_debug(enabled):
    global DEBUG_ENABLED
    DEBUG_ENABLED = enabled
    return gr.update(visible=enabled)


# -------------------------------------------------
# Reset full session (chat + debug)
# -------------------------------------------------
def reset_session():
    DEBUG_LOGS.clear()
    return [], "", ""


# -------------------------------------------------
# Clear ONLY debug logs
# -------------------------------------------------
def clear_debug_logs():
    DEBUG_LOGS.clear()
    return ""


# -------------------------------------------------
# Gradio UI definition (Hospital Operations)
# -------------------------------------------------
with gr.Blocks() as ui:

    # ===== Header =====
    gr.Markdown("""
    ## 🏥 Hospital Operations Dashboard  
    **Internal Control System – Read Only**
    
    **Role:** Hospital Operations Manager  
    **Scope:** Inventory • Machines • Vendors • Patients  
    """)

    # ===== Top Controls =====
    with gr.Row():
        debug_toggle = gr.Checkbox(
            value=True,
            label="🛠 Enable Debug Mode"
        )
        reset_btn = gr.Button("🔄 Fresh Session")

    # ===== Main Layout =====
    with gr.Row():

        # -------- Left: Chat --------
        with gr.Column(scale=3):

            chat = gr.Chatbot(
                type="messages",
                height=480,
                label="📋 Operational Query Log"
            )

            box = gr.Textbox(
                label="Operations Query",
                placeholder="Ask about inventory, machines, vendors, patients…"
            )

            btn = gr.Button("▶ Execute Query")

        # -------- Right: Debug Panel --------
        with gr.Column(scale=2):

            with gr.Row():
                gr.Markdown("### 🧠 System Execution Trace (Live)")
                clear_debug_btn = gr.Button("🧹 Clear", size="sm")

            debug_panel = gr.Textbox(
                lines=26,
                interactive=False,
                visible=True
            )

    # ===== Toggle Debug Visibility =====
    debug_toggle.change(
        fn=toggle_debug,
        inputs=debug_toggle,
        outputs=debug_panel
    )

    # ===== Execute Query =====
    btn.click(
        fn=ask_chat,
        inputs=[box, chat],
        outputs=[chat, box]
    ).then(
        fn=get_debug_logs,
        outputs=debug_panel
    )

    # ===== Clear Debug Only =====
    clear_debug_btn.click(
        fn=clear_debug_logs,
        outputs=debug_panel
    )

    # ===== Fresh Session (Chat + Debug) =====
    reset_btn.click(
        fn=reset_session,
        outputs=[chat, box, debug_panel]
    )

print("DEBUG UI ready")
ui.launch()


DEBUG UI ready
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


[DEBUG] User Query → hi 
[DEBUG] Non-database query → routing to chat LLM
DEBUG: Sending to LLM-B
[DEBUG] User Query → How many units of normal saline are available
[DEBUG] Database query detected → generating intent
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["quantity"], "filters": [{"column": "item_name", "op": "LIKE", "value": "Normal Saline"}], "group_by": [], "limit": null}
DEBUG fuzzy 'Normal Saline' → 'Normal Saline (0.9%)' (76.15384615384616)
DEBUG normalized: Normal Saline → Normal Saline (0.9%)
DEBUG FINAL INTENT: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Normal Saline (0.9%)'}], 'group_by': [], 'limit': None}
[DEBUG] Executing query on table → inventory
DEBUG running query: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Normal Saline (0.9%)'}], 'group_by': [

C:\Users\ayush\AppData\Local\Temp\ipykernel_8524\3587826986.py:43: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result = result[result[col].astype(str).str.contains(val, case=False, na=False)]


[DEBUG] User Query → Do we have insulin in stock?
[DEBUG] Database query detected → generating intent
DEBUG: Sending to LLM-A
DEBUG RAW INTENT: {"action": "select", "table": "inventory", "columns": ["quantity"], "filters": [{"column": "item_name", "op": "LIKE", "value": "Insulin"}], "group_by": [], "limit": null}
DEBUG fuzzy 'Insulin' → 'Insulin Regular' (77.14285714285715)
DEBUG normalized: Insulin → Insulin Regular
DEBUG FINAL INTENT: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Insulin Regular'}], 'group_by': [], 'limit': None}
[DEBUG] Executing query on table → inventory
DEBUG running query: {'action': 'select', 'table': 'inventory', 'columns': ['quantity'], 'filters': [{'column': 'item_name', 'op': 'LIKE', 'value': 'Insulin Regular'}], 'group_by': [], 'limit': None}
DEBUG applying LIKE filter: item_name contains 'Insulin Regular'
DEBUG result rows: 1
[DEBUG] Data found → sending result to explanatio